<a href="https://colab.research.google.com/github/ddchristiantan/DSND_Term2/blob/master/MLZoomcamp_homewor3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Input

url = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [3]:
df = pd.read_csv(url)

In [4]:
df = df[['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']]

In [5]:
df.head().T

,0,1,2,3,4
Make,BMW,BMW,BMW,BMW,BMW
Model,1 Series M,1 Series,1 Series,1 Series,1 Series
Year,2011,2011,2011,2011,2011
Engine HP,335.0,300.0,300.0,230.0,230.0
Engine Cylinders,6.0,6.0,6.0,6.0,6.0
Transmission Type,MANUAL,MANUAL,MANUAL,MANUAL,MANUAL
Vehicle Style,Coupe,Convertible,Coupe,Coupe,Convertible
highway MPG,26,28,28,28,28
city mpg,19,19,20,18,18
MSRP,46135,40650,36350,29450,34500


In [6]:
data = df.copy()

In [7]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [8]:
data = data.fillna(0)

In [9]:
data.rename(columns = {'msrp':'price'}, inplace = True)


In [10]:
data.head().T

,0,1,2,3,4
make,BMW,BMW,BMW,BMW,BMW
model,1 Series M,1 Series,1 Series,1 Series,1 Series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
transmission_type,MANUAL,MANUAL,MANUAL,MANUAL,MANUAL
vehicle_style,Coupe,Convertible,Coupe,Coupe,Convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500


In [11]:
data.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
dtype: object

Question 1

In [12]:
data.transmission_type.mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

Question 2

In [13]:
correlation_matrix = data[['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'price']].corr()

correlation_matrix_no_diag = correlation_matrix.where(~pd.DataFrame(np.eye(correlation_matrix.shape[0]), dtype=bool))
max_corr = correlation_matrix_no_diag.abs().max().max()
max_corr_features = [(index, column) for index in correlation_matrix_no_diag.index for column in correlation_matrix_no_diag.columns if abs(correlation_matrix_no_diag.at[index, column]) == max_corr]

correlation_matrix, max_corr_features


(                      year  engine_hp  engine_cylinders  highway_mpg  \
 year              1.000000   0.338714         -0.040708     0.258240   
 engine_hp         0.338714   1.000000          0.774851    -0.415707   
 engine_cylinders -0.040708   0.774851          1.000000    -0.614541   
 highway_mpg       0.258240  -0.415707         -0.614541     1.000000   
 city_mpg          0.198171  -0.424918         -0.587306     0.886829   
 price             0.227590   0.650095          0.526274    -0.160043   
 
                   city_mpg     price  
 year              0.198171  0.227590  
 engine_hp        -0.424918  0.650095  
 engine_cylinders -0.587306  0.526274  
 highway_mpg       0.886829 -0.160043  
 city_mpg          1.000000 -0.157676  
 price            -0.157676  1.000000  ,
 [])

In [14]:
correlation_matrix_no_diag = correlation_matrix.copy()
np.fill_diagonal(correlation_matrix_no_diag.values, 0)


max_corr_value = correlation_matrix_no_diag.abs().max().max()
max_corr_features = [(index, col) for col in correlation_matrix_no_diag.columns for index in correlation_matrix_no_diag.index if abs(correlation_matrix_no_diag.at[index, col]) == max_corr_value]

max_corr_value, max_corr_features


(0.8868294962591425,
 [('city_mpg', 'highway_mpg'), ('highway_mpg', 'city_mpg')])

In [15]:
mean_price = data['price'].mean()

data['above_average'] = (data['price'] > mean_price).astype(int)


In [51]:
data.dtypes

make                  object
model                 object
year                  object
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int64
dtype: object

In [54]:
data['year'] = data['year'].astype(str)
data['engine_hp'] = data['engine_hp'].astype(str)
data['engine_cylinders'] = data['engine_cylinders'].astype(str)

In [58]:
categorical = list(data.dtypes[data.dtypes == 'object'].index)
numerical = ['highway_mpg','city_mpg','price']

In [16]:
data.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0


In [27]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)



In [29]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [30]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [31]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_val.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

Question 3

In [40]:
from sklearn.metrics import mutual_info_score

In [34]:
choices = ['make', 'model','transmission_type', 'vehicle_style',]

In [42]:
def mutual_info_x_score(series):
  return mutual_info_score(series, df_full_train.above_average)

In [43]:
mi = df_full_train[choices].apply(mutual_info_x_score)
mi.sort_values(ascending=False)

model                0.460994
make                 0.238724
vehicle_style        0.083390
transmission_type    0.020884
dtype: float64

Question 4

In [45]:
from sklearn.feature_extraction import DictVectorizer

In [59]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical+numerical].to_dict(orient='records')
df_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical+numerical].to_dict(orient='records')
df_val = dv.transform(val_dict)

In [60]:
from sklearn.linear_model import LogisticRegression

In [61]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

model.fit(df_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [62]:
model.intercept_[0]

-9.658766624208772e-05

In [63]:
y_pred = model.predict_proba(df_val)[:,1]

In [64]:
from sklearn.metrics import accuracy_score

In [68]:
accuracy_score(y_val, y_pred >= 0.99)

0.9924464960134285